In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoConfig
import torch
from customModel import customBertForSequenceClassification, customRobertaForSequenceClassification,customGPT2ForSequenceClassification, customElectraForSequenceClassification
import numpy as np
from frontModelCustom import id2labelData_labels
from tqdm import tqdm
import pickle

/home/leesk/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
# Model_path = './best_model/e_60_t_40_g_52'
Model_path = './my_awesome_model/checkpoint-6000'
Model_path = 'SangGank/my-front-model'
# Model_path = 'SangGank/my-front-model'
tokenizer = AutoTokenizer.from_pretrained(Model_path)
model = customRobertaForSequenceClassification.from_pretrained(Model_path)

In [19]:
with open('labels.pkl','rb') as f:
    emotion_labels=pickle.load(f)
    tempo_labels=pickle.load(f)
    genre_labels=pickle.load(f)

In [20]:
emotion , tempo, genre = id2labelData_labels('labels.pkl')

def logits2Label(logits):
    emotion_id = int(logits[0].detach().argmax())
    tempo_id = int(logits[1].detach().argmax())
    genre_id = int(logits[2].detach().argmax())
    return emotion[emotion_id], tempo[tempo_id], genre[genre_id]

In [24]:
# text= 'smooth and sultry, with a catchy and memorable piano riff that sets the tone for the rest of the track'
text= "My rival die"
inputs = tokenizer(text, return_tensors='pt')

result = model(**inputs).logits
logits2Label(result)

('anger', 'Moderato', 'Groove')

{'input_ids': tensor([[    0, 11377, 14891,  1462,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
(tensor([[-1.0156, -7.6180, -0.0323, -3.1650, -5.8199, -7.3814, -3.6764, -5.1977,
         -4.6829, -6.2326]], grad_fn=<AddmmBackward0>), tensor([[ 0.9522, -0.8574, -6.7239, -3.0107]], grad_fn=<AddmmBackward0>), tensor([[-2.9883, -5.4740, -7.4847, -6.3331, -6.8134, -4.6010, -7.7749,  1.1946,
         -1.2455, -6.0410]], grad_fn=<AddmmBackward0>))


('excitement', 'Allegro', 'Classical')

('excitement', 'Allegro', 'Classical')

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv("./genre_11_tempo_4.csv")

In [5]:
data2 = data.copy()
emotion_data = data2.groupby('emotion').sample(frac=0.05, random_state=42)
tempo_data = data2.groupby('tempo(category)').sample(frac=0.05, random_state=42)
genre_data = data2.groupby('genre').sample(frac=0.05, random_state=42)
index_total = set(emotion_data.index) | set(tempo_data.index) | set(genre_data.index)
valid_data = data2.iloc[list(index_total)]
train_data = data2.drop(list(index_total)).sample(frac=1, random_state=42)

In [6]:
valid_data.emotion.unique()

array(['excitement', 'nostalgia', 'anger', 'love', 'sadness', 'happiness',
       'loneliness', 'anticipation', 'calmness', 'gratitude'],
      dtype=object)

In [23]:
valid_data['caption'].values[1]


"catchy and upbeat, with a memorable chorus that showcases Dayne's powerful vocals."

In [21]:
len(valid_data['caption'].values)

1419

In [26]:
valid_data.iloc[1]['caption']

"catchy and upbeat, with a memorable chorus that showcases Dayne's powerful vocals."

In [33]:
li_emotion = []
li_tempo = []
li_genre = []
for i in tqdm(range(len(valid_data))):
    text = valid_data.iloc[i]['caption']
    inputs = tokenizer(text, return_tensors='pt')
    result = model(**inputs).logits
    em, tm,ger = logits2Label(result)
    li_emotion.append(em)
    li_tempo.append(tm)
    li_genre.append(ger)

100%|██████████| 1419/1419 [01:14<00:00, 18.94it/s]


In [34]:
valid_data2 = valid_data.copy()
valid_data2['pred_emotion'] =li_emotion
valid_data2['pred_tempo'] =li_tempo
valid_data2['pred_genre'] =li_genre
valid_data2.to_csv('./predict.csv',index=False)


In [8]:
# for i in valid_data['caption'].values:
#     inputs = tokenizer(i, return_tensors='pt')
#     result = model(**inputs).logits
#     emo, tem, gen = logits2Label(result)
#     if gen != 'Rock':
#         print(i)

In [9]:
model.config

BertConfig {
  "_name_or_path": "./my_awesome_model/checkpoint-2400",
  "architectures": [
    "customRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "admiration",
    "1": "amusement",
    "2": "anger",
    "3": "annoyance",
    "4": "approval",
    "5": "caring",
    "6": "confusion",
    "7": "curiosity",
    "8": "desire",
    "9": "disappointment",
    "10": "disapproval",
    "11": "disgust",
    "12": "embarrassment",
    "13": "excitement",
    "14": "fear",
    "15": "gratitude",
    "16": "grief",
    "17": "joy",
    "18": "love",
    "19": "nervousness",
    "20": "optimism",
    "21": "pride",
    "22": "realization",
    "23": "relief",
    "24": "remorse",
    "25": "sadness",
    "26": "surprise",
    "27": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size

In [10]:
text= 'smooth and sultry, with a catchy and memorable piano riff that sets the tone for the rest of the track'
# text= 'very sad song give me'
inputs = tokenizer(text, return_tensors='pt')
print(inputs)

result = model(**inputs).logits
print(result)

{'input_ids': tensor([[    0,  9426, 22571,     8,   579,  6070,  1506,     6,    19,    10,
         33200,     8, 10132, 13305, 31457,    14,  3880,     5,  6328,    13,
             5,  1079,     9,     5,  1349,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}
(tensor([[-1.6020, -0.6746, -2.0802, -4.0002, -1.7004, -4.7404, -1.8420, -4.7288,
         -5.0525, -4.5234]], grad_fn=<AddmmBackward0>), tensor([[-0.1990, -0.8512, -2.9543, -1.1062]], grad_fn=<AddmmBackward0>), tensor([[-3.0486, -1.6718, -2.6524,  0.9791, -1.8430, -3.2592, -2.8961, -2.8547,
         -3.8543, -5.0551, -3.6929]], grad_fn=<AddmmBackward0>))


In [34]:
emotion , tempo, genre = id2labelData_labels('labels.pkl')

def logits2Label(logits):
    emotion_id = int(logits[0].detach().argmax())
    tempo_id = int(logits[1].detach().argmax())
    genre_id = int(logits[2].detach().argmax())
    return emotion[emotion_id], tempo[tempo_id], genre[genre_id]

In [35]:
logits2Label(result)

('excitement', 'Allegro', 'Jazz')

In [23]:
import pickle
label_data_path ='./labels.pkl'
with open(label_data_path,'rb') as f:
        emotion_labels=pickle.load(f)
        tempo_labels=pickle.load(f)
        genre_labels=pickle.load(f)

print(tempo_labels)

['Allegro' 'Moderato' 'Andante' 'Presto']


In [24]:
emotion , tempo, genre = id2labelData_labels('labels.pkl')

def logits2Label(logits):
    emotion_id = int(logits[0].detach().argmax())
    tempo_id = int(logits[1].detach().argmax())
    genre_id = int(logits[2].detach().argmax())
    return emotion[emotion_id], tempo[tempo_id], genre[genre_id]

In [2]:
import torch
x= torch.tensor([[1,2,3,4,5]])

x1 = torch.tensor([[1,2,3,4,5]])
x+x1

tensor([[ 2,  4,  6,  8, 10]])

In [25]:
logits2Label(result)

('love', 'Moderato', 'Pop')

In [ ]:
import torch

print(torch.__version__)

2.2.1+cu121
